# Train the Model

## Import Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from models import Encoder, Decoder

from pathlib import Path
from DatasetInterface import MSCOCOInterface
import json


## Load Dataset Interface and DataLoader

In [2]:
# load vocab
with open('vocabulary/idx_to_string.json') as json_file:
    idx_to_string_json = json.load(json_file)
        
idx_to_string = dict()
for key in idx_to_string_json:
    idx_to_string[int(key)] = idx_to_string_json[key]
    
with open('vocabulary/string_to_index.json') as json_file:
    string_to_index = json.load(json_file)


In [3]:
# to boost the performence of CUDA use:
# torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
root = Path('Data')
#imgs_path = root/'images'/'train2017'
imgs_path = root/'train2017'
#captions_path = root/'annotations'/'captions_train2017.json'
captions_path = root/'annotations_trainval2017'/'annotations'/'captions_train2017.json'

interface_params = {
    'imgs_path': imgs_path,
    'captions_path': captions_path,
    'freq_threshold': 1,
    'sequence_length': 20,
    'caps_per_img': 10,
    'idx_to_string': idx_to_string,
    'string_to_index': string_to_index
}

coco_interface_train = MSCOCOInterface(**interface_params)
print(f"Length of the interface: {len(coco_interface_train)}")

Length of the interface: 591435


In [5]:
batch_size = 1
shuffle = True
train_loader = torch.utils.data.DataLoader(coco_interface_train, batch_size=batch_size, shuffle=shuffle)

In [6]:
root = Path('Data')
#imgs_path = root/'images'/'train2017'
imgs_path = root/'val2017'
#captions_path = root/'annotations'/'captions_train2017.json'
captions_path = root/'annotations_trainval2017'/'annotations'/'captions_val2017.json'

interface_params = {
    'imgs_path': imgs_path,
    'captions_path': captions_path,
    'freq_threshold': 1,
    'sequence_length': 20,
    'caps_per_img': 10,
    'idx_to_string': idx_to_string,
    'string_to_index': string_to_index
}

coco_interface_val = MSCOCOInterface(**interface_params)
print(f"Length of the interface: {len(coco_interface_val)}")

Length of the interface: 25000


In [7]:
batch_size = 1
shuffle = False
val_loader = torch.utils.data.DataLoader(coco_interface_val, batch_size=batch_size, shuffle=shuffle)

## Parameters

In [8]:
embed_size = 512
hidden_size = 512
vocab_size = len(coco_interface_train.idx_to_string)
num_layers = 1
total_epochs = 1

## Encoder and Decoder

In [9]:
encoder = Encoder(embed_size=embed_size, pretrained=False, model_weight_path="./model/resnet152_model.pth")
decoder = Decoder(embed_size=embed_size, hidden_size=hidden_size, vocab_size=vocab_size, num_layers=num_layers)

encoder.to(device)
decoder.to(device)
print("##########READY##########")

##########READY##########


## Train

In [12]:
loss = nn.CrossEntropyLoss(ignore_index=coco_interface_train.string_to_index["<PAD>"])
params = list(decoder.parameters()) + list(encoder.embed.parameters())
optimizer = torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08)

In [13]:
idx, images, captions = next(iter(train_loader))
images, captions = images.to(device), captions.to(device)

In [14]:
features = encoder(images)
outputs = decoder(features, captions)

In [15]:
outputs

tensor([[[ 0.0150,  0.0329,  0.0113,  ..., -0.0115,  0.0165, -0.0461],
         [ 0.1050,  0.0595,  0.0192,  ..., -0.1173, -0.0631,  0.1219],
         [ 0.0180,  0.1344, -0.1162,  ..., -0.0894, -0.2035,  0.0119],
         ...,
         [ 0.0345, -0.0225,  0.0688,  ...,  0.0698, -0.0841, -0.0700],
         [-0.0076, -0.0465,  0.0538,  ...,  0.0460,  0.0270, -0.0686],
         [-0.0329, -0.0548,  0.0555,  ...,  0.0332,  0.0830, -0.0623]]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [16]:
l = loss(outputs.view(-1, vocab_size), captions.contiguous().view(-1))
l

tensor(10.2008, device='cuda:0', grad_fn=<NllLossBackward0>)

In [17]:
idxx = decoder.get_predict(features, 20)

In [20]:
idxx = idxx[0].cpu().numpy()

In [21]:
sampled_caption = []
for word_id in idxx:
    word = coco_interface_train.idx_to_string[word_id]
    sampled_caption.append(word)
    if word == "<END>":
        break
sampled_caption

['celll',
 'acouple',
 'symposium',
 'chocolate',
 'pontoon',
 'remot',
 'toppers',
 '1933',
 'flicking',
 'occupying',
 'sullenly',
 'willow',
 'tucked',
 'vitamin',
 'q',
 'mallard',
 'unbaked',
 'after',
 'bedside',
 'stuffs']

In [15]:
training_loss = []
validation_loss = []
for epoch in range(total_epochs):
    
    train_epoch_loss = 0
    val_epoch_loss = 0
    
    encoder.zero_grad()
    decoder.zero_grad()
    
    encoder.train()
    decoder.train()
    
    for idx, batch in enumerate(train_loader):
        print(idx)
        idx, images, captions = batch
        images, captions = images.to(device), captions.to(device)
        
        features = encoder(images)
        outputs = decoder(features, captions)
        
        l = loss(outputs.view(-1, vocab_size), captions.contiguous().view(-1))
        
        train_epoch_loss += l.item()
        
        l.backward()
        optimizer.step()
    
    train_epoch_loss = train_epoch_loss/len(train_loader)
    training_loss.append(train_epoch_loss)
    
    encoder.eval()
    decoder.eval()
    
    for idx, batch in enumerate(val_loader):
        idx, images, captions = batch
        images, captions = images.to(device), captions.to(device)
        
        features = encoder(images)
        outputs = decoder(features, captions)
        
        l = loss(outputs.view(-1, vocab_size), captions.contiguous().view(-1))
        
        val_epoch_loss += l.item()
    
    val_epoch_loss = val_epoch_loss/len(val_loader)
    training_loss.append(val_epoch_loss)
    
    
    print(f"Training Loss: {train_epoch_loss}, Validation Loss: {val_epoch_loss}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 